# Multi-Crop Leaf Disease Detection
This notebook handles **data preprocessing, model training, and evaluation** using MobileNetV2.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

## 🔍 Load PlantVillage Metadata

In [ ]:
data_path = '../data/PlantVillage_metadata.csv'
df = pd.read_csv(data_path)
df.head()

## 🧪 Image Preprocessing using ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='../data/sample_images',
    x_col='filename',
    y_col='label',
    subset='training',
    batch_size=4,
    seed=42,
    shuffle=True,
    class_mode='categorical',
    target_size=(224, 224))

val_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='../data/sample_images',
    x_col='filename',
    y_col='label',
    subset='validation',
    batch_size=4,
    seed=42,
    shuffle=True,
    class_mode='categorical',
    target_size=(224, 224))

## 🤖 Build & Train MobileNetV2 Model

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(train_generator, validation_data=val_generator, epochs=3)

## 📊 Model Evaluation & Training History

In [ ]:
os.makedirs('../model', exist_ok=True)
os.makedirs('../screenshots', exist_ok=True)
model.save('../model/leaf_model.keras')

print(history.history.keys())

plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.savefig('../screenshots/accuracy_loss_curve.png')
plt.show()

## 🔁 Confusion Matrix for Validation Set

In [ ]:
y_true = val_generator.classes
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=val_generator.class_indices.keys(), 
            yticklabels=val_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('../screenshots/confusion_matrix.png')
plt.show()